In [190]:
import sys # for automation and parallelization: set manual to false when run by a launcher
manual, scenario = (True, 'project_office') if 'ipykernel' in sys.argv[0] else (False, sys.argv[1])

# START

In [191]:
import sys
sys.path.insert(0, r'../../../quetzal')

from quetzal.model import stepmodel
from quetzal.io import excel
import os
import numpy as np
import pandas as pd
if manual:
    %matplotlib inline

In [192]:
parameter_frame = pd.read_excel('../../inputs/parameters.xlsx')

In [193]:
parameter_frame.drop(['description', 'unit', 'by'], axis=1, errors='ignore', inplace=True)

# IN

In [194]:
training_folder = '../../'

In [195]:
sm = stepmodel.read_zippedpickles(r'../../model/{scen}/networks'.format(scen=scenario))
import geopandas as gpd


zone_to_transit: 100%|█████████████████████████████████████████████████████████████████| 33/33 [00:00<00:00, 54.04it/s]


In [196]:
sm = stepmodel.read_zippedpickles(r'../../model/{scen}/networks'.format(scen=scenario))
import geopandas as gpd
zones = gpd.read_file(r'../../inputs/zones/zones_with_population_2040.geojson')
sm.zones = zones.set_index('id')

zone_to_transit: 100%|█████████████████████████████████████████████████████████████████| 33/33 [00:00<00:00, 54.10it/s]


In [198]:
# the scenario specific variables are read from the parameter file
var = excel.read_var(file='../../inputs/parameters.xlsx', scenario=scenario)

In [ ]:
boarding_time = var['general']['boarding_time']

In [ ]:
import rasterio
import rasterio.plot

In [ ]:
raster = rasterio.open(r'../../inputs/tiff/20137.tiff')

In [ ]:
fontsize=14
if manual:
    ax = sm.plot(
        'links', basemap_url=None, figsize=[20, 20], color='green',
        basemap_raster=raster, north_arrow=True, scalebar=True, linewidth=0.5)

# SPEED, HEADWAY AND FARE

In [ ]:
sm.links['route_type'] = 'bajaj'

if var['general']['code']=='base':
    sm.links['mode'] = 'bajaj'

In [ ]:
sm.links.columns

In [ ]:
headways = var['headway'].to_dict()
vehicle_capacities = var['vehicle_capacities'].to_dict()

sm.links['headway'] = np.nan
sm.links['headway'].fillna(sm.links['mode'].apply(lambda t: headways.get(t, np.nan) ), inplace=True)
sm.links['headway'] = sm.links['headway'].astype(int)
sm.links['vehicle_capacity'] = sm.links['mode'].apply(vehicle_capacities.get)

#if var['general']['code']=='base':
#    sm.links['headway'] = np.nan
#    sm.links['headway'].fillna(sm.links['route_type'].apply(lambda t: headways.get(t, np.nan) ), inplace=True)
#    sm.links['vehicle_capacity'] = sm.links['route_type'].apply(vehicle_capacities.get)
#else:
#    sm.links['headway'] = sm.links['mode'].apply(headways.get)
#    sm.links['vehicle_capacity'] = sm.links['mode'].apply(vehicle_capacities.get)

In [ ]:
sm.links['per_hour'] = 3600 / sm.links['headway'] 
sm.links['capacity'] = (sm.links['vehicle_capacity'] * sm.links['per_hour']).astype(int)

In [ ]:
#sm.links['length'] = sm.links['road_length']
walk_speed = 4
sm.road_links['walk_speed'] = float(var['speed']['walk_on_road']) # km / h
sm.road_links['speed'] = float(var['speed']['car']) # km / h
sm.road_links['walk_time'] = sm.road_links['length'] / sm.road_links['walk_speed'] / 1000 * 3600

In [ ]:
sm.road_links['time'] = sm.road_links['length'] / sm.road_links['speed'] / 1000 * 3600

In [ ]:
sm.links['length'] = sm.links['geometry'].apply(lambda g: g.length)

In [ ]:
sm.links['time']=sm.links['length']/float(var['speed']['car']) / 1000 * 3600
sm.links['time']*=1.05

#Show the benefit of a BRT

if var['general']['code']!='base':
    for l in sm.links.index:
        if sm.links['mode'][l]=='brt':
            sm.links['time'][l]=sm.links['time'][l]*var['speed']['car']/var['speed']['brt']

# WALKMODEL

In [ ]:
sm = sm.change_epsg(epsg=20137, coordinates_unit='meter')

In [ ]:
kwargs = var['preparation_footpaths'].to_dict()
sm.preparation_footpaths(**kwargs)
print('preparation_footpaths:', kwargs)

kwargs = var['preparation_ntlegs'].to_dict()
sm.preparation_ntlegs(zone_to_road=True, zone_to_transit=True, **kwargs)
print('preparation_ntlegs:', kwargs)

sm.zone_to_transit['length'] = sm.zone_to_transit['geometry'].apply(lambda g: g.length)
sm.zone_to_transit['time'] = sm.zone_to_transit['length'] / sm.zone_to_transit['speed'] * 3600 / 1000

In [ ]:
if manual:
    sm.plot('road_to_transit', figsize=[20, 20], basemap_raster=raster, title='Road to Transit')
    sm.plot('zone_to_road', figsize=[20, 20], basemap_raster=raster, title='Zone to Road')

In [ ]:
sm._add_type_prefixes( prefixes={'links': 'link_', 'zone_to_road': 'ztr_','road_to_transit': 'rtt_'})
sm.zones = sm.zones.reindex(set(sm.zone_to_transit['a']).intersection(set(sm.zones.index)))
sm.zones = sm.zones.reindex(set(sm.zone_to_road['a']).intersection(set(sm.zones.index)))

In [ ]:
sm.links.head()

# PT PATHFINDER  | optimal strategy

In [ ]:
sm.links = sm.links.loc[sm.links['a'] != sm.links['b']]
sm.step_strategy_finder(walk_on_road=True, boarding_time=boarding_time)

In [ ]:
origin, destination ='k2_diredawa_university', 'k8_misrak_buna_board'

if manual:
    plot = sm.plot_strategy(
        origin, destination, road=True, cmap='Reds', ax=None , walk_on_road=True,
        figsize=(15, 10), legend='right', legend_kwds={'orientation': 'vertical', 'label': 'probability'},
        basemap_raster=raster, scalebar=True, north_arrow=True
    )

In [ ]:
#origin, destination ='zone_157', 'k2_kebele_02' 
os.environ["PATH"] += os.pathsep + r'C:\Users\mgomezdelarosa\graphviz-2.38\release\bin/'
os.environ["PATH"] += os.pathsep + r'C:\Users\qchasserieau\dev\graphviz-2.38\release\bin/'
sm.display_aggregated_edges( origin, destination, ranksep=0.5,rankdir='LR', irrelevant_nodes=set(sm.road_nodes.index))

# ROAD PATHFINDER | all or nothing

In [ ]:
sm.step_road_pathfinder(all_or_nothing=True)

In [ ]:
if not os.path.isdir(r'../../model/{scen}'.format(scen=scenario)):
    os.mkdir(r'../../model/%s' % scenario)
sm.to_zippedpickles(r'../../model/{scen}/aon_pathfinder'.format(scen=scenario))  

In [ ]:
end_of_notebook